Inspired from https://www.kaggle.com/kimtaegwan/what-s-your-cv-method

In [1]:
import numpy as np 
import pandas as pd 

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold, StratifiedKFold
from tqdm import tqdm_notebook
import os
import gc

import lightgbm as lgb
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

In [2]:
directory = './'
# os.listdir(directory)

# Data

In [3]:
sample_submission = pd.read_csv(directory+'/sample_submission.csv')
df_train = pd.read_csv(directory+'/train.csv')
weather_train = pd.read_csv(directory+'/weather_train.csv')
building_metadata = pd.read_csv(directory+'/building_metadata.csv')

print('df_train_shape :' , df_train.shape)
print('weather_train_shape :' , weather_train.shape)
print('building_metadata_shape :' , building_metadata.shape)

df_train_shape : (20216100, 4)
weather_train_shape : (139773, 9)
building_metadata_shape : (1449, 6)


In [4]:
del df_train
df_train = pd.read_feather('train_cleanup_001.feather')
df_train.columns

Index(['building_id', 'meter', 'timestamp', 'meter_reading'], dtype='object')

In [5]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
building_metadata.primary_use = le.fit_transform(building_metadata.primary_use)



building_metadata['cnt_building_per_site'] = building_metadata.groupby(['site_id']).building_id.transform(lambda x: x.size)
building_metadata['cnt_building_per_site_prim'] = building_metadata.groupby(['site_id', 'primary_use']).building_id.transform(lambda x: x.size)
building_metadata['sqr_mean_per_site'] = building_metadata.groupby(['site_id', ]).square_feet.transform('median')
building_metadata['sqr_mean_per_prim_site'] = building_metadata.groupby(['site_id', 'primary_use']).square_feet.transform('median')

building_metadata.head()

site_id  building_id  primary_use  square_feet  year_built  floor_count  \
0        0            0            0         7432      2008.0          NaN   
1        0            1            0         2720      2004.0          NaN   
2        0            2            0         5376      1991.0          NaN   
3        0            3            0        23685      2002.0          NaN   
4        0            4            0       116607      1975.0          NaN   

   cnt_building_per_site  cnt_building_per_site_prim  sqr_mean_per_site  \
0                    105                          30            52957.0   
1                    105                          30            52957.0   
2                    105                          30            52957.0   
3                    105                          30            52957.0   
4                    105                          30            52957.0   

   sqr_mean_per_prim_site  
0                 72175.5  
1                 72175.5  
2                 72175.5  
3                 72175.5  
4                 72175.5

In [6]:
df_train = df_train.merge(building_metadata, on='building_id', how='left')
df_train = df_train.merge(weather_train, on=['site_id', 'timestamp'], how='left')
del weather_train
gc.collect()

print('df_train_merge shape: ',df_train.shape)

df_train_merge shape:  (19695046, 20)


In [7]:
df_train.columns

Index(['building_id', 'meter', 'timestamp', 'meter_reading', 'site_id',
       'primary_use', 'square_feet', 'year_built', 'floor_count',
       'cnt_building_per_site', 'cnt_building_per_site_prim',
       'sqr_mean_per_site', 'sqr_mean_per_prim_site', 'air_temperature',
       'cloud_coverage', 'dew_temperature', 'precip_depth_1_hr',
       'sea_level_pressure', 'wind_direction', 'wind_speed'],
      dtype='object')

In [8]:
df_train.shape

(19695046, 20)

In [9]:
df_train.columns

Index(['building_id', 'meter', 'timestamp', 'meter_reading', 'site_id',
       'primary_use', 'square_feet', 'year_built', 'floor_count',
       'cnt_building_per_site', 'cnt_building_per_site_prim',
       'sqr_mean_per_site', 'sqr_mean_per_prim_site', 'air_temperature',
       'cloud_coverage', 'dew_temperature', 'precip_depth_1_hr',
       'sea_level_pressure', 'wind_direction', 'wind_speed'],
      dtype='object')

# feature engneering

In [10]:
#drop col
unimportant_col=['sea_level_pressure', 'wind_direction','wind_speed']
df_train.drop(unimportant_col,axis=1,inplace=True)

#time variable
df_train["timestamp"] = pd.to_datetime(df_train["timestamp"])
df_train["hour"] = df_train["timestamp"].dt.hour
df_train["weekday"] = df_train["timestamp"].dt.weekday
df_train["month"] = df_train["timestamp"].dt.month
df_train['year_built'] = df_train['year_built']-1900
df_train['square_feet'] = np.log1p(df_train['square_feet'])

dates_range = pd.date_range(start='2015-12-31', end='2019-01-01')
us_holidays = calendar().holidays(start=dates_range.min(), end=dates_range.max())

df_train['is_holiday'] = (df_train['timestamp'].dt.date.astype('datetime64').isin(us_holidays)).astype(np.int8)
del df_train["timestamp"]

df_train['meter_reading'] = np.log1p(df_train['meter_reading'])

# Memory reduce

In [11]:
#Based on this great kernel https://www.kaggle.com/arjanso/reducing-dataframe-memory-size-by-65
def reduce_mem_usage(df):
    start_mem_usg = df.memory_usage().sum() / 1024**2 
    print("Memory usage of properties dataframe is :",start_mem_usg," MB")
    NAlist = [] # Keeps track of columns that have missing values filled in. 
    for col in df.columns:
        if df[col].dtype != object:  # Exclude strings            
            # Print current column type
            print("******************************")
            print("Column: ",col)
            print("dtype before: ",df[col].dtype)            
            # make variables for Int, max and min
            IsInt = False
            mx = df[col].max()
            mn = df[col].min()
            print("min for this col: ",mn)
            print("max for this col: ",mx)
            # Integer does not support NA, therefore, NA needs to be filled
            if not np.isfinite(df[col]).all(): 
                NAlist.append(col)
                df[col].fillna(mn-1,inplace=True)  
                   
            # test if column can be converted to an integer
            asint = df[col].fillna(0).astype(np.int64)
            result = (df[col] - asint)
            result = result.sum()
            if result > -0.01 and result < 0.01:
                IsInt = True            
            # Make Integer/unsigned Integer datatypes
            if IsInt:
                if mn >= 0:
                    if mx < 255:
                        df[col] = df[col].astype(np.uint8)
                    elif mx < 65535:
                        df[col] = df[col].astype(np.uint16)
                    elif mx < 4294967295:
                        df[col] = df[col].astype(np.uint32)
                    else:
                        df[col] = df[col].astype(np.uint64)
                else:
                    if mn > np.iinfo(np.int8).min and mx < np.iinfo(np.int8).max:
                        df[col] = df[col].astype(np.int8)
                    elif mn > np.iinfo(np.int16).min and mx < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    elif mn > np.iinfo(np.int32).min and mx < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                    elif mn > np.iinfo(np.int64).min and mx < np.iinfo(np.int64).max:
                        df[col] = df[col].astype(np.int64)    
            # Make float datatypes 32 bit
            else:
                df[col] = df[col].astype(np.float32)
            
            # Print new column type
            print("dtype after: ",df[col].dtype)
            print("******************************")
    # Print final result
    print("___MEMORY USAGE AFTER COMPLETION:___")
    mem_usg = df.memory_usage().sum() / 1024**2 
    print("Memory usage is: ",mem_usg," MB")
    print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
    return df, NAlist

In [12]:
df_train ,_ = reduce_mem_usage(df_train)

Memory usage of properties dataframe is : 3024.0081844329834  MB
******************************
Column:  building_id
dtype before:  int64
min for this col:  0
max for this col:  1448
dtype after:  uint16
******************************
******************************
Column:  meter
dtype before:  int64
min for this col:  0
max for this col:  3
dtype after:  uint8
******************************
******************************
Column:  meter_reading
dtype before:  float64
min for this col:  0.0
max for this col:  16.902211829285342
dtype after:  float32
******************************
******************************
Column:  site_id
dtype before:  int64
min for this col:  0
max for this col:  15
dtype after:  uint8
******************************
******************************
Column:  primary_use
dtype before:  int64
min for this col:  0
max for this col:  15
dtype after:  uint8
******************************
******************************
Column:  square_feet
dtype before:  float64
min for t

In [13]:
category = []
category = ['month','hour','meter','weekday','primary_use','site_id','building_id']
for col in category :
    df_train[col] = df_train[col].astype('category')



# Model

In [14]:
# force the model to use the weather data instead of dates, to avoid overfitting to the past history
features = [col for col in df_train.columns if col not in ['meter_reading', 'year', 'month', 'day', 'index','precip_depth_1_hr', 'floor_count','cloud_coverage',]]
target = 'meter_reading'
features

['building_id',
 'meter',
 'site_id',
 'primary_use',
 'square_feet',
 'year_built',
 'cnt_building_per_site',
 'cnt_building_per_site_prim',
 'sqr_mean_per_site',
 'sqr_mean_per_prim_site',
 'air_temperature',
 'dew_temperature',
 'hour',
 'weekday',
 'is_holiday']

In [15]:
df_train = df_train.reset_index(drop=True)

folds = 3
seed = 42
kf = KFold(n_splits=folds, shuffle=False, random_state=seed)
# oof_pred = np.zeros(train.shape[0])  # out of fold predictions
models = []
oof = np.zeros(len(df_train))
## stratify data by building_id
print('Light GBM Model')
for fold_, (trn_idx, val_idx)  in enumerate(kf.split(df_train, df_train['meter_reading'])):
        tr_x, tr_y = df_train[features].iloc[trn_idx], df_train[target][trn_idx]
        vl_x, vl_y = df_train[features].iloc[val_idx], df_train[target][val_idx]
        print({'train size':len(tr_x), 'eval size':len(vl_x)})
        print('fold : ',fold_)
        tr_data = lgb.Dataset(tr_x, label=tr_y)
        vl_data = lgb.Dataset(vl_x, label=vl_y) 
        reg= lgb.LGBMRegressor(n_estimators=6000,
                                learning_rate=0.05,
                                feature_fraction=0.7,
                                subsample=0.8,
                                num_leaves=40,
                                metric='rmse')
        reg.fit(tr_x, tr_y, eval_set=[(tr_x, tr_y),(vl_x, vl_y)],early_stopping_rounds=200,verbose=500)
        oof[val_idx] = reg.predict(df_train.iloc[val_idx][features])
        models.append(reg)
        gc.collect()
    
print('oof_RMSE : ' ,np.sqrt(mean_squared_error(oof, df_train['meter_reading'])))

Light GBM Model
{'train size': 13130030, 'eval size': 6565016}
fold :  0
Training until validation scores don't improve for 200 rounds
[500]	training's rmse: 0.814237	valid_1's rmse: 1.04033
Early stopping, best iteration is:
[639]	training's rmse: 0.798143	valid_1's rmse: 1.03437
{'train size': 13130031, 'eval size': 6565015}
fold :  1
Training until validation scores don't improve for 200 rounds
[500]	training's rmse: 0.84595	valid_1's rmse: 0.951267
[1000]	training's rmse: 0.820338	valid_1's rmse: 0.929156
[1500]	training's rmse: 0.802016	valid_1's rmse: 0.913816
[2000]	training's rmse: 0.789833	valid_1's rmse: 0.907855
[2500]	training's rmse: 0.779536	valid_1's rmse: 0.900958
[3000]	training's rmse: 0.771633	valid_1's rmse: 0.897706
[3500]	training's rmse: 0.765633	valid_1's rmse: 0.896168
[4000]	training's rmse: 0.759776	valid_1's rmse: 0.894577
Early stopping, best iteration is:
[4283]	training's rmse: 0.757203	valid_1's rmse: 0.89402
{'train size': 13130031, 'eval size': 6565015

In [16]:
print('oof_RMSE : ' ,np.sqrt(mean_squared_error(oof, df_train['meter_reading'])))

oof_RMSE :  0.9800632722387246


# predict

In [17]:
df_test = pd.read_csv('test.csv')
weather_test = pd.read_csv('weather_test.csv')

print('df_test_shape :' , df_test.shape)
print('weather_test_shape :' , weather_test.shape)

df_test_shape : (41697600, 4)
weather_test_shape : (277243, 9)


In [18]:
df_test = df_test.merge(building_metadata, on='building_id', how='left')
df_test = df_test.merge(weather_test, on=['site_id', 'timestamp'], how='left')
# del weather_test
# del building_metadata
gc.collect()

print('df_test_merge shape: ',df_test.shape)

df_test_merge shape:  (41697600, 20)


In [19]:
df_test ,_ = reduce_mem_usage(df_test)

Memory usage of properties dataframe is : 6680.67626953125  MB
******************************
Column:  row_id
dtype before:  int64
min for this col:  0
max for this col:  41697599
dtype after:  uint32
******************************
******************************
Column:  building_id
dtype before:  int64
min for this col:  0
max for this col:  1448
dtype after:  uint16
******************************
******************************
Column:  meter
dtype before:  int64
min for this col:  0
max for this col:  3
dtype after:  uint8
******************************
******************************
Column:  site_id
dtype before:  int64
min for this col:  0
max for this col:  15
dtype after:  uint8
******************************
******************************
Column:  primary_use
dtype before:  int64
min for this col:  0
max for this col:  15
dtype after:  uint8
******************************
******************************
Column:  square_feet
dtype before:  int64
min for this col:  283
max for this

In [20]:
#drop col
unimportant_col=['sea_level_pressure', 'wind_direction','wind_speed']
df_test.drop(unimportant_col,axis=1,inplace=True)

#time variable
df_test["timestamp"] = pd.to_datetime(df_test["timestamp"])
df_test["hour"] = df_test["timestamp"].dt.hour
df_test["weekday"] = df_test["timestamp"].dt.weekday
df_test["month"] = df_test["timestamp"].dt.month
df_test['year_built'] = df_test['year_built']-1900
df_test['square_feet'] = np.log1p(df_test['square_feet'])


df_test['is_holiday'] = (df_test['timestamp'].dt.date.astype('datetime64').isin(us_holidays)).astype(np.int8)

del df_test["timestamp"]

In [21]:
# category = ['month','hour','meter','weekend','primary_use','site_id','building_id']
for col in category :
    df_test[col] = df_test[col].astype('category')

In [23]:
# split test data into batches
set_size = len(df_test)
iterations = 50
batch_size = set_size // iterations

print(set_size, iterations, batch_size)
assert set_size == iterations * batch_size

41697600 50 833952


In [24]:
df_train.head()

building_id meter  meter_reading site_id primary_use  square_feet  \
0         105     0       3.190624       1           0    10.832181   
1         106     0       0.318163       1           0     8.589514   
2         106     3       0.000000       1           0     8.589514   
3         107     0       5.171529       1           0    11.487947   
4         108     0       4.524668       1           0    11.309352   

   year_built  floor_count  cnt_building_per_site  cnt_building_per_site_prim  \
0         255            5                     51                          22   
1         255            4                     51                          22   
2         255            4                     51                          22   
3         105           10                     51                          22   
4          13            5                     51                          22   

   sqr_mean_per_site  sqr_mean_per_prim_site  air_temperature  cloud_coverage  \
0            57673.0                 64707.5              3.8             255   
1            57673.0                 64707.5              3.8             255   
2            57673.0                 64707.5              3.8             255   
3            57673.0                 64707.5              3.8             255   
4            57673.0                 64707.5              3.8             255   

   dew_temperature  precip_depth_1_hr hour weekday month  is_holiday  
0              2.4                 -2    0       4     1           1  
1              2.4                 -2    0       4     1           1  
2              2.4                 -2    0       4     1           1  
3              2.4                 -2    0       4     1           1  
4              2.4                 -2    0       4     1           1

In [25]:
df_test# = df_test.reset_index(drop=True)

row_id building_id meter site_id primary_use  square_feet  \
0                0           0     0       0           0     8.913685   
1                1           1     0       0           0     7.908755   
2                2           2     0       0           0     8.589886   
3                3           3     0       0           0    10.072639   
4                4           4     0       0           0    11.666573   
...            ...         ...   ...     ...         ...          ...   
41697595  41697595        1444     0      15           1     9.884305   
41697596  41697596        1445     0      15           0     8.366138   
41697597  41697597        1446     0      15           1     9.329545   
41697598  41697598        1447     0      15           4    10.301458   
41697599  41697599        1448     0      15           6    11.432496   

          year_built  floor_count  cnt_building_per_site  \
0                108            0                    105   
1                104            0                    105   
2                 91            0                    105   
3                102            0                    105   
4                 75            0                    105   
...              ...          ...                    ...   
41697595          14            0                    124   
41697596       65535            0                    124   
41697597          97            0                    124   
41697598         101            0                    124   
41697599         101            0                    124   

          cnt_building_per_site_prim  sqr_mean_per_site  \
0                                 30            52957.0   
1                                 30            52957.0   
2                                 30            52957.0   
3                                 30            52957.0   
4                                 30            52957.0   
...                              ...                ...   
41697595                          16            65831.0   
41697596                          46            65831.0   
41697597                          16            65831.0   
41697598                          29            65831.0   
41697599                          19            65831.0   

          sqr_mean_per_prim_site  air_temperature  cloud_coverage  \
0                        72175.5        17.799999               4   
1                        72175.5        17.799999               4   
2                        72175.5        17.799999               4   
3                        72175.5        17.799999               4   
4                        72175.5        17.799999               4   
...                          ...              ...             ...   
41697595                 72423.0       -29.100000             255   
41697596                 82926.0       -29.100000             255   
41697597                 72423.0       -29.100000             255   
41697598                 61439.0       -29.100000             255   
41697599                 53303.0       -29.100000             255   

          dew_temperature  precip_depth_1_hr hour weekday month  is_holiday  
0               11.700000                 -2    0       6     1           0  
1               11.700000                 -2    0       6     1           0  
2               11.700000                 -2    0       6     1           0  
3               11.700000                 -2    0       6     1           0  
4               11.700000                 -2    0       6     1           0  
...                   ...                ...  ...     ...   ...         ...  
41697595       -32.599998                 -2    7       2     5           0  
41697596       -32.599998                 -2    7       2     5           0  
41697597       -32.599998                 -2    7       2     5           0  
41697598       -32.599998                 -2    7       2     5           0  
41697599       -32

In [26]:
meter_reading = []
for i in tqdm_notebook(range(iterations)):
    pos = i*batch_size
    fold_preds = [np.expm1(model.predict(df_test[features].iloc[pos : pos+batch_size])) for model in models]
    meter_reading.extend(np.mean(fold_preds, axis=0))

print(len(meter_reading))
assert len(meter_reading) == set_size


41697600


In [27]:
sample_submission['meter_reading'] = np.clip(meter_reading, a_min=0, a_max=None) # clip min at zero
sample_submission.to_csv('submission_whatsyourcv3_0052_trncl.csv.gz',index=False,compression='gzip', float_format='%.4f', chunksize=25000)
sample_submission.head(15)

row_id  meter_reading
0        0     168.497114
1        1      66.425652
2        2       7.806381
3        3     293.877462
4        4    1237.251613
5        5      14.085194
6        6     105.518307
7        7     355.469406
8        8     324.870742
9        9     329.351272
10      10      46.226389
11      11       9.625583
12      12    1084.573330
13      13     308.759776
14      14     214.547362

In [34]:
sample_submission['meter_reading'] = np.clip(meter_reading, a_min=0, a_max=None) # clip min at zero
sample_submission.to_csv('submission_whatsyourcv3_0052_trncl.csv.gz',index=False,compression='gzip', float_format='%.4f', chunksize=25000)
sample_submission.head(15)

row_id  meter_reading
0        0     172.709355
1        1      77.085027
2        2      10.646963
3        3     294.440831
4        4    1210.374520
5        5      12.590222
6        6     109.740265
7        7     414.242261
8        8     815.539733
9        9     326.339801
10      10      47.053357
11      11      62.579348
12      12    1101.249671
13      13     409.675993
14      14     245.894101

In [28]:
!kaggle competitions submit -c ashrae-energy-prediction -f submission_whatsyourcv3_0052_trncl.csv.gz -m "lgbm old style"

100%|████████████████████████████████████████| 260M/260M [00:05<00:00, 52.9MB/s]
Successfully submitted to ASHRAE - Great Energy Predictor III